In [ ]:
import pandas as pd
import numpy as np

logSZ = pd.read_csv(r'/mnt/data/20201203/zs_7201_postmdgw_20201203_10.0.101.56/Logs_full/mdLog_SZ_20201203_0903.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
logSZ1 = pd.read_csv(r'/mnt/data/20201203/zs_7201_postmdgw_20201203_10.0.101.57/Logs_full/mdLog_SZ_20201203_0903.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
print("original data source type: ")
print(logSZ["source"].unique())
print("new data source type: ")
print(logSZ1["source"].unique())
print(logSZ["time"].unique())
print(logSZ1["time"].unique())

pd.set_option('max_columns', 200)
logSZ1["time"] = logSZ1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))

display(logSZ[(logSZ["StockID"] == 1) & (logSZ["time"] >= 112000000) & (logSZ["time"] <= 113000000)])
display(logSZ1[(logSZ1["StockID"] == 1) & (logSZ1["time"] >= 112000000) & (logSZ1["time"] <= 113000000)])

In [ ]:
data1 = logSZ[(logSZ["cum_volume"] > 0) & (logSZ["source"] == 31) & (logSZ['time'] <= 150000000)]
data2 = logSZ1[(logSZ1["cum_volume"] > 0) & (logSZ1["source"] == 31) & (logSZ1['time'] <= 150000000)]
###
columns = ["StockID", "cum_volume", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades"]
###

# columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
#            "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
#            "ask4q", "ask5q", "openPrice", "numTrades"]

data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())
print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
data1_1['cum_amount'] = data1_1['cum_amount'].round(2)
data2_1['cum_amount'] = data2_1['cum_amount'].round(2)
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    display(n2-n1)

In [ ]:
# speed check
test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

In [ ]:
import pandas as pd
import random
import numpy as np
import pickle

TradeLog1 = pd.read_csv(r'/mnt/data/20201203/zs_6801_20201203_168.26.0.161/Logs_full/mdTradeLog_20201203_0900.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum", "mdSource"]]
OrderLog1 = pd.read_csv(r'/mnt/data/20201203/zs_6801_20201203_168.26.0.161/Logs_full/mdOrderLog_20201203_0900.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty", "mdSource"]]
TradeLog = pd.read_csv(r'/mnt/data/20201203/zs_6801_20201203_168.26.0.160/Logs_full/mdTradeLog_20201203_0859.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum", "mdSource"]]
OrderLog = pd.read_csv(r'/mnt/data/20201203/zs_6801_20201203_168.26.0.160/Logs_full/mdOrderLog_20201203_0859.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty", "mdSource"]]
stocks = list(set(OrderLog1["SecurityID"].unique()) & set(OrderLog["SecurityID"].unique()))
display(len(stocks))
OrderLogSZ = OrderLog[OrderLog["SecurityID"].isin(stocks)]
OrderLogSZ1 = OrderLog1[OrderLog1["SecurityID"].isin(stocks)]
del OrderLog
del OrderLog1
TradeLogSZ = TradeLog[TradeLog["SecurityID"].isin(stocks)]
TradeLogSZ1 = TradeLog1[TradeLog1["SecurityID"].isin(stocks)]
del TradeLog
del TradeLog1
OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
OrderLogSZ1["OrderType"] = OrderLogSZ1["OrderType"].apply(lambda x: str(x))
print(len(OrderLogSZ["SecurityID"].unique()))
print(len(OrderLogSZ1["SecurityID"].unique()))
print(len(TradeLogSZ["SecurityID"].unique()))
print(len(TradeLogSZ1["SecurityID"].unique()))
print(OrderLogSZ["mdSource"].unique())
print(OrderLogSZ1["mdSource"].unique())
print(TradeLogSZ["mdSource"].unique())
print(TradeLogSZ1["mdSource"].unique())
print(OrderLogSZ["exchId"].unique())
print(OrderLogSZ1["exchId"].unique())
print(TradeLogSZ["exchId"].unique())
print(TradeLogSZ1["exchId"].unique())

In [ ]:
TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

# TradeLogSZ = TradeLogSZ[TradeLogSZ['TransactTime'] >= TradeLogSZ1['TransactTime'].min()]
columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
# columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradePrice", "TradeQty", "BidApplSeqNum","OfferApplSeqNum"]

# TradeLogSZ1 = TradeLogSZ1.drop_duplicates(columns, keep='first')

re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)
    
    
# speed test
re = re[~re["sequenceNo_x"].isnull()]
re = re[~re["sequenceNo_y"].isnull()]
diff = re["clockAtArrival_y"] - re["clockAtArrival_x"]
re["diff"] = re["clockAtArrival_y"] - re["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))
print(sum(diff == 0)/len(diff))

In [ ]:
columns = ["TransactTime", "ApplSeqNum","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]

# OrderLogSZ = OrderLogSZ[OrderLogSZ['TransactTime'] >= OrderLogSZ1['TransactTime'].min()]
ree = pd.merge(OrderLogSZ, OrderLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = ree["sequenceNo_x"].count()
n2 = ree["sequenceNo_y"].count()
len1 = len(ree)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(ree[np.isnan(ree["sequenceNo_y"])])
    print(len(ree[np.isnan(ree["sequenceNo_y"])]))
    print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("test is complete, baseline is not complete:")
    display(ree[np.isnan(ree["sequenceNo_x"])])
    print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

# speed test
ree = ree[~ree["sequenceNo_x"].isnull()]
ree = ree[~ree["sequenceNo_y"].isnull()]
ree["diff"] = ree["clockAtArrival_y"] - ree["clockAtArrival_x"]
diff = ree["clockAtArrival_y"] - ree["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))